<a href="https://colab.research.google.com/github/Nick-P-Uconn/ECE-6141-Term-Project/blob/main/ECE_6141_Term_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# 📦 ENVIRONMENT SETUP
# ============================================================
!pip install open_spiel torch matplotlib numpy pandas --quiet

import torch, pyspiel, numpy as np, matplotlib.pyplot as plt, pandas as pd
print("✅ Libraries installed successfully")

# Optional: Mount Google Drive to save outputs
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 8.9 MB/s eta 0:00:00
✅ Libraries installed successfully
Mounted at /content/drive


In [ ]:
#import Open Spiel Backgammon and torch
import pyspiel, torch, torch.nn as nn, torch.optim as optim
import numpy as np, random, pandas as pd
from collections import deque
from pathlib import Path
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# configuration varibles
n_episodes = 200000
discount_rate = 0.99 #gamma\
#Inteval to test against random opponent to ensure improving win rate
eval_interval = 2000
# Numebr of test Games per evaluation
rand_test_Num_Games = 100

#Rand_opening is used to ensure that over traning various early/mid game states are encountered and are also rewarded
#this ensure the model has some training on all levels of the game
#caps the amount of moves madde to ensure that the game is not in end game to start
rand_opening_move_cap = 8

# Exploration schedules
#epsilon and Tau are set high early to implore the model to make mistakes while learning
#as time goes on Epsilon deccrease resulting in minmal changes to the model
max_epsilon, min_epsilon, epsilon_decay_rate = 1, 0.05, 75000   # epsilon

#as time goes on decrease Tau to ensure the model only picks the most win likely
#scenario in accordance with its training
#reduce to 1e-3  or less to produce an effective argmax for training
max_tau, min_tau = 8, 0.1                    # softmax temperature



#import the game of backgammon fromm Open SPiel
game = pyspiel.load_game("backgammon")

#code obtained from open spiel documentation
def reset_to_playable_state():
    #sets the game to the intial board state
    s = game.new_initial_state()
    while s.is_chance_node():
        # chance_outcomes() -> list[(action_id, prob)]
        a = random.choice(s.chance_outcomes())[0]
        s.apply_action(a)
    return s


def new_state(rand_opening=True):
   #calls the intial board state
    s = reset_to_playable_state()
    #if in traning perform random moves
    if rand_opening:
        for _ in range(random.randint(0, rand_opening_move_cap)):
            #break if game has ended somehow
            if s.is_terminal(): break
            #roll dice
            if s.is_chance_node():
                a = random.choice(s.chance_outcomes())[0]
                s.apply_action(a)
                continue
            #checks if there is a legal move based on the dice roll
            acts = s.legal_actions()
            #if a legal move exists choose one of the random actions
            if acts:
                s.apply_action(random.choice(acts))
            else:
                break
    return s

# Player-0 observation length
input_dim = len(reset_to_playable_state().observation_tensor(0))

# ---------------------------- Value Network ----------------------------------
class ValueNet(nn.Module):
    def __init__(self, dim, h=256):
        super().__init__()
        #creates a nerual network with 2 hidden layers one with 256 neurons the other with
        #128 neurons activation function for each is a rectified linear unit
        self.net = nn.Sequential(
            nn.Linear(dim, h), nn.ReLU(),
            nn.Linear(h, h//2), nn.ReLU(),
            #final layer acts as the state output essentially what move should we make
            nn.Linear(h//2, 1)
        )
    def forward(self, x): return self.net(x)

value_net = ValueNet(input_dim)
#use an adam optimizer with a learning rate of .001 for the nerual network
optimizer = optim.Adam(value_net.parameters(), lr=1e-4)
#Loss=(1/N)∑​(Vθ​(si​)−targeti​)^2
criterion = nn.MSELoss()


def get_eps_tau(ep):
    #determines the epsilon and tau values for a given episode
    #based on the decay rate and episode
    #Note max function ensure neither epsilon or taur ever get to low in a given episode
    epsilon = max(min_epsilon, max_epsilon - (max_epsilon - min_epsilon)*(ep/ epsilon_decay_rate))
    tau = max(min_tau, max_tau - (max_tau - min_tau)*(ep/epsilon_decay_rate))
    return epsilon, tau

def select_action(state, ep, softmax=True):
    epsilon, tau = get_eps_tau(ep)

    # If we're at a chance node, resolve it first (shouldn't happen in training loop,
    # but safe-guard in case of randomized openings landing on a chance node)
    while state.is_chance_node():
        a = random.choice(state.chance_outcomes())[0]
        state.apply_action(a)

    legal = state.legal_actions()
    if not legal:
        return None, epsilon, tau

    if random.random() < epsilon:
        return random.choice(legal), epsilon, tau

    vals = []
    for a in legal:
        child = state.child(a)
        obs = torch.tensor(child.observation_tensor(0), dtype=torch.float32)
        with torch.no_grad():
            vals.append(value_net(obs).item())
    if softmax:
        probs = torch.softmax(torch.tensor(vals)/tau, dim=0)
        idx = torch.multinomial(probs, 1).item()
    else:
        idx = int(np.argmax(vals))
    return legal[idx], epsilon, tau

# ---------------------------- Episode Rollout --------------------------------
def play_episode(ep, gamma=discount_rate):
    # start new game
    s = new_state(rand_opening=True)

    total_r = 0.0
    n_steps = 0
    epsilon = tau = None

    # while game is going
    while not s.is_terminal():
        # select the move (epsilon / tau schedule)
        a, epsilon, tau = select_action(s, ep, softmax=True)

        # Handle no-legal-actions (pass) edge case
        if a is None:
            acts = s.legal_actions()
            if acts:
                s.apply_action(acts[0])
            continue

        # current state observation
        obs = torch.tensor(s.observation_tensor(0), dtype=torch.float32)

        # apply chosen action
        s.apply_action(a)

        # reward from this step
        r = s.rewards()[0]
        total_r += r

        # next state observation
        nxt_obs = torch.tensor(s.observation_tensor(0), dtype=torch.float32)
        done = s.is_terminal()

        # ------------- TD(0) UPDATE (online) -------------
        with torch.no_grad():
            target_val = r if done else r + gamma * value_net(nxt_obs).item()

        target = torch.tensor([target_val], dtype=torch.float32)

        pred = value_net(obs)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    return total_r, epsilon, tau, loss.item()


# ---------------------------- Evaluation -------------------------------------
def random_action(state):
    #aqpplys a random action when evaluating against the random player
    while state.is_chance_node():
        a = random.choice(state.chance_outcomes())[0]
        state.apply_action(a)
    acts = state.legal_actions()
    return random.choice(acts) if acts else None

def evaluate_vs_random(agent_net, n_games=50):
    wins = 0
    for _ in range(n_games):
        s = new_state(rand_opening=False)
        while not s.is_terminal():
            while s.is_chance_node():
                a = random.choice(s.chance_outcomes())[0]
                s.apply_action(a)
            if s.current_player() == 0:
                legal = s.legal_actions()
                if not legal:
                    # pass if environment says so
                    s.apply_action(s.legal_actions()[0]) if s.legal_actions() else None
                else:
                    vals = []
                    for a in legal:
                      #as this is the player choose the actions with best probaility according to RL model
                        child = s.child(a)
                        obs = torch.tensor(child.observation_tensor(0), dtype=torch.float32)
                        #note model returns best moves in a read only mode do not want to allow updates
                        #based on performance against purley randomized opponent
                        with torch.no_grad():
                            vals.append(agent_net(obs).item())
                    a = legal[int(np.argmax(vals))]
                    s.apply_action(a)
            else:
              #random player thus use randomized action
                a = random_action(s)
                if a is not None:
                    s.apply_action(a)
        #if the model won add one to the win column
        if s.rewards()[0] > 0:
            wins += 1
    return wins / n_games


#note this is effectivley equivelant to polaying rnadom just using a save state
#to continue to show agents improvment through training
def evaluate_net_vs_net(net_A, net_B, n_games=50):
    wins_A = 0
    for g in range(n_games):
        s = new_state(rand_opening=False)

        # alternate player order for fairness
        if g % 2 == 0:
            p0, p1 = net_A, net_B
        else:
            p0, p1 = net_B, net_A

        while not s.is_terminal():
            while s.is_chance_node():
                a = random.choice(s.chance_outcomes())[0]
                s.apply_action(a)

            legal = s.legal_actions()
            if not legal:
                break

            # pick network for the player to move
            net = p0 if s.current_player() == 0 else p1

            vals = []
            for a in legal:
                child = s.child(a)
                obs = torch.tensor(child.observation_tensor(0), dtype=torch.float32)
                with torch.no_grad():
                    vals.append(net(obs).item())
            a = legal[int(np.argmax(vals))]
            s.apply_action(a)

        r0 = s.rewards()[0]
        #determine if the network under training one if so then we are doing a good job technically
        if g % 2 == 0:
            if r0 > 0: wins_A += 1
        else:
            if r0 < 0: wins_A += 1

    return wins_A / n_games


losses, rewards, epsilons, taus, winrates = [], [], [], [], []
rw = deque(maxlen=50)

for ep in range(1, n_episodes + 1):

    # play one episode and update online during it
    final_r, epsilon, tau, avg_loss = play_episode(ep, discount_rate)

    # log epsilon/tau once per episode
    if epsilon is None or tau is None:
        epsilon, tau = get_eps_tau(ep)

    epsilons.append(epsilon)
    taus.append(tau)

    # one TD_Loss per episode: average loss over that episode's moves
    losses.append(avg_loss)
    rewards.append(final_r)
    rw.append(final_r)




    # if a certain amount of games has been played then call the evaluation
    if (ep % eval_interval) == 0 or ep == 10 or (ep < 500 and ep % (eval_interval/4) == 0):

        # at ep = 50000, save the baseline model once
        if ep == 50000:
            Path("checkpoints").mkdir(exist_ok=True)
            torch.save(value_net.state_dict(), "checkpoints/value_net_ep50000.pth")

        # choose evaluation opponent
        if ep < 50000:

            wr = evaluate_vs_random(value_net, n_games=rand_test_Num_Games)
            opp_label = "Random"
        else:
            # at/after 50000: evaluate
            baseline = ValueNet(input_dim)
            baseline.load_state_dict(torch.load("checkpoints/value_net_ep50000.pth"))
            wr = evaluate_net_vs_net(value_net, baseline, n_games=rand_test_Num_Games)
            opp_label = "EP50000 Model"

        winrates.append((ep, wr))
        # prints the parameters used in this set of win-rate evaluation
        print(f"Ep {ep:4d}/{n_episodes} | ε={epsilons[-1]:.3f} τ={taus[-1]:.2f} | "
              f"Loss={losses[-1]:.4f} | AvgR(50)={np.mean(rw):+.3f} | "
              f"WinRate vs {opp_label} = {wr:.2f}")




print("Training complete")

# ---------------------------- Save Model ------------------------------------
out = Path("checkpoints")
out.mkdir(exist_ok=True)

model_path = out/"value_net.pth"

torch.save({
    "model_state": value_net.state_dict(),
    "opt_state": optimizer.state_dict()
}, model_path)

# save CSV
df = pd.DataFrame({
    "Episode": np.arange(1, n_episodes + 1),
    "TD_Loss": losses,
    "Reward": rewards,
    "Epsilon": epsilons,
    "Tau": taus,
    "SmoothedReward": pd.Series(rewards).rolling(50).mean()
})

for ep_wr, wr in winrates:
    df.loc[df["Episode"] == ep_wr, "WinRate"] = wr

df.to_csv(out/"training_log.csv", index=False)

# ---------------------------- Download Model ---------------------------------
from google.colab import files
files.download(str(model_path))
files.download("checkpoints/training_log.csv")



Using device: cuda
Ep   10/200000


KeyboardInterrupt: 